In [1]:
import folium
from folium import Element
import pandas as pd
from string import Template

In [2]:
df = pd.read_csv("log_sheet.csv").fillna("").astype(str)

In [3]:
df = df.rename(columns = {"Name*": "Name"})

for col in ["Metres", "Scott", "Matthew"]:
    df[col] = df[col].apply(lambda x: x.split(".")[0])

In [4]:
table_css = """<style>
              table {
            
                    border-collapse: collapse;
                    font-family: Arial, sans-serif;
                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
                    background: #fff;
                    table-layout: fixed;
                  }

                    th {
                        border: 1px solid #ddd;
                        background-color: #f7f7f7;
                        text-align: left;
                        font-weight: 600;
                        height: 70px;
                      }
            
                td {
                    border: 1px solid #ddd;
                    text-align: left;
                    height: 70px;
                  }

            </style>"""

In [5]:
table_search = """
<script>
  document.addEventListener("DOMContentLoaded", function () {
    const searchInput = document.getElementById('searchInput');
    const rows = document.querySelectorAll('#mountainTable tbody tr');
    searchInput.addEventListener('input', function () {
      const filter = searchInput.value.toLowerCase();
      rows.forEach(row => {
        const nameText = row.cells[0].textContent.toLowerCase();
        row.style.display = nameText.includes(filter) ? '' : 'none';
      });
    });
  });
</script>
"""

In [6]:
row_template = Template("""<tr><td><button onclick="window.map.setView([$lat, $long], 13)">$name</button></td></tr>""")

In [7]:
table_rows = ""
for _, row in df.iterrows():

    table_rows += row_template.substitute({'name': row["Name"],
                            'lat': row["Latitude"],
                            'long': row["Longitude"],
                            })


In [8]:
table_css = """
<style>
  table {
    border-collapse: collapse;
    font-family: Arial, sans-serif;
    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    background: #fff;
    table-layout: fixed;
  }
  th {
    border: 1px solid #ddd;
    background-color: #f7f7f7;
    text-align: left;
    font-weight: 600;
    height: auto;
  }
  td {
    border: 1px solid #ddd;
    text-align: left;
    height: auto;
  }
</style>
"""

table_script = """
<script>
  document.addEventListener("DOMContentLoaded", function () {
    const searchInput = document.getElementById('searchInput');
    const rows = document.querySelectorAll('#mountainTable tbody tr');
    searchInput.addEventListener('input', function () {
      const filter = searchInput.value.toLowerCase();
      rows.forEach(row => {
        const nameText = row.cells[0].textContent.toLowerCase();
        row.style.display = nameText.includes(filter) ? '' : 'none';
      });
    });
  });
</script>
"""

sidebar_html = f"""
            <div id="sidebar" style="
              position: fixed;
              top: 0;
              left: 0;
              width: 170px;
              height: 100vh;
              background: #ffffff;
              overflow-y: auto;
              padding: 20px 15px;
              box-shadow: 2px 0 10px rgba(0, 0, 0, 0.1);
              z-index: 1000;
              font-family: 'Segoe UI', Tahoma, sans-serif;
              font-size: 14px;
              line-height: 1.6;
              color: #333;
              border-right: 1px solid #ddd;
              transition: transform 0.3s ease;
            ">
              <table id="mountainTable">
                <colgroup><col style="width: 100px"></colgroup>
                <thead>
                  <tr>
                    <th>
                      Name<br>
                      <input type="text" id="searchInput" placeholder="🔍" style="width: 100px">
                    </th>
                  </tr>
                </thead>
                <tbody>
                  {table_rows}
                </tbody>
              </table>
            </div>
    
            <!-- Toggle Button -->
            <button id="toggleSidebar" style="
              position: fixed;
              top: 10px;
              left: 180px;
              z-index: 1100;
              padding: 5px 10px;
              background-color: #ffffff;
              border: 1px solid #ccc;
              border-radius: 4px;
              cursor: pointer;
              font-size: 12px;
            ">Hide table</button>
            
            """

scotland_center = [56.4907, -4.2026]
m = folium.Map(location=scotland_center, zoom_start=6, zoom_control=False)

map_div_id = m.get_name()
map_css = f"""
<style>
  #{map_div_id} {{
    position: absolute;
    top: 0;
    left: 180;
    right: 0;
    bottom: 0;
    z-index: 0;
  }}
</style>
"""

assign_map_js = f"""
<script>
  window.onload = function() {{
    window.map = {map_div_id};
    window.panMap = function(lat, lng) {{
      window.map.setView([lat, lng], 10);
    }};
  }};
</script>
"""

assign_toggle_js = """
<script>
  const sidebar = document.getElementById('sidebar');
  const toggleButton = document.getElementById('toggleSidebar');

  toggleButton.addEventListener('click', function () {
    if (sidebar.style.transform === 'translateX(-170px)') {
      sidebar.style.transform = 'translateX(0)';
      toggleButton.style.left = '180px';
      toggleButton.textContent = 'Hide table';
    } else {
      sidebar.style.transform = 'translateX(-170px)';
      toggleButton.style.left = '10px';
      toggleButton.textContent = 'Show table';
    }
  });
</script>
"""

m.get_root().header.add_child(Element(map_css))
m.get_root().header.add_child(Element(table_css))
m.get_root().html.add_child(Element(sidebar_html))
m.get_root().header.add_child(Element(table_script))
m.get_root().html.add_child(Element(assign_map_js))
m.get_root().html.add_child(Element(assign_toggle_js))

In [9]:
matthew_bag = folium.CustomIcon(icon_image="https://raw.githubusercontent.com/mprallison/munro_bagger/main/images/matthew.png",
                            icon_size=(14, 14),
                            icon_anchor=(12, 14)
                            )

scott_bag = folium.CustomIcon(icon_image="https://raw.githubusercontent.com/mprallison/munro_bagger/main/images/scott.png",
                            icon_size=(14, 14),
                            icon_anchor=(2, 14)
                            )

peak = folium.CustomIcon(icon_image="https://raw.githubusercontent.com/mprallison/munro_bagger/main/images/peak.png",
                            icon_size=(14, 14),
                            icon_anchor=(7, 7)
                            )

In [10]:
bag_icons = {"Matthew": matthew_bag,
             "Scott": scott_bag,
            }

In [11]:
def add_peak(row):

    name = row["Name"]
    height = row["Metres"]
    coords = [row["Latitude"], row["Longitude"]]

    label = "<br>".join([name, f"height: {height}m"])

    icon = peak

    folium.Marker(
                location = coords,
                tooltip = label,
                icon = icon
                ).add_to(m)

    return

In [12]:
def add_bag(row):

    year = row[person]
    icon = bag_icons[person]
    coords = [row["Latitude"], row["Longitude"]]
    
    folium.Marker(
    location = coords,
    tooltip = year,
    icon = icon
    ).add_to(m)

    return

In [13]:
for _, row in df.iterrows():
    add_peak(row)

In [14]:
for person in ["Matthew", "Scott"]:
    
    for _, row in df[df[person] != ""].iterrows():
        add_bag(row)

In [15]:
# Save
m.save("munro_map.html")